### Baseline для хакатона Rutube по задаче "Теггирование видео"

В рамках данного ноутбука мы рассмотрим наивный подход к решению поставленной задачи: векторный поиск навания видео в базе векторов тегов.

В конце есть пример получения sample_submission.csv - пример файла, который нужно загрузить на лидерборд.


In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from tqdm.autonotebook import tqdm
import numpy as np 
import faiss

#### Берем данные с id видео и его названием, также загружаем иерархические теги

In [2]:
data = pd.read_csv("train_data_categories.csv", index_col=0)[['video_id', 'title']]
taxonomy = pd.read_csv("IAB_tags.csv")

print(data.columns)
print(data.head(5))

print(taxonomy.head(5))
print(taxonomy.columns)

Index(['video_id', 'title'], dtype='object')
  video_id                                              title
0  video_0  Шульский VS Дамил. Пояс. Мурад VS Черкасов. Га...
1  video_1  Тарасов VS Хейбати. Намитов VS Тарасов за пояс...
2  video_2  Мурад и Черкасов кровавое побоище. Мага Исма в...
3  video_3  Шульский vs Вагабов. Тарасов vs Стоун.Сушист Ф...
4  video_4  Такой расклад. Эфир 54 | Таро | Ответы на ваши...
  Уровень 1 (iab)         Уровень 2 (iab)      Уровень 3 (iab)
0       Транспорт                     NaN                  NaN
1       Транспорт  Типы кузова автомобиля                  NaN
2       Транспорт  Типы кузова автомобиля  Грузовой автомобиль
3       Транспорт  Типы кузова автомобиля                Седан
4       Транспорт  Типы кузова автомобиля            Универсал
Index(['Уровень 1 (iab)', 'Уровень 2 (iab)', 'Уровень 3 (iab)'], dtype='object')


#### Для создания эмбеддинга берем русскоязычный Берт и загружаем в sentence transformer, который позволяет создавать эмбеддинг для всего предложения и сам обрезает его до максимально возможного числа токенов

In [3]:
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence', )
dim = 768 # размер вектора эмбеддинга

No sentence-transformers model found with name DeepPavlov/rubert-base-cased-sentence. Creating a new one with mean pooling.
/home/kivanova/download_videos/venv_tagging/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### Создаем эмбеддинги для названий видео

In [4]:
data['title_vector'] = data['title'].apply(lambda l: model.encode(l, convert_to_tensor=True).cpu().numpy())

#### Создаем векторы для тегов:
 Для каждого 1 уровня иерархии в отдельности и для следующих уровней формата уровень 1: уровень 2: уровень 3 

In [5]:
def get_tags():
    tags = {}
    for i, row in tqdm(taxonomy.iterrows()):
        if isinstance(row['Уровень 1 (iab)'], str):
            tags[row['Уровень 1 (iab)']] = model.encode(row['Уровень 1 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 2 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
        if isinstance(row['Уровень 3 (iab)'], str):
            tags[row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)']] = model.encode(row['Уровень 1 (iab)']+ ": "+row['Уровень 2 (iab)']+": "+row['Уровень 3 (iab)'], convert_to_tensor=True).cpu().numpy()#.tolist()
    return tags

tags = get_tags()
tags_list = list(tags.keys())
vectors = np.array(list(tags.values()))

611it [00:12, 48.02it/s]


### Создаем векторную базу faiss для эффективного векторного поиска

In [6]:
index = faiss.index_factory(dim, "Flat", faiss.METRIC_INNER_PRODUCT)
print(index.ntotal)
index.add(vectors)
print(index.ntotal)

0
610


#### Смотрим несколько получившихся примеров 
Генерим по 3 близких предсказания для каждого названия видео

In [7]:
topn = 3
scores, predictions = index.search(np.array(data['title_vector'].to_list()[:10]), topn)
for j, i in enumerate(predictions):
    print("SCORES", scores[j])
    print("PREDICTION_by_title", np.array(tags_list)[predictions[j]])
    print("SAMPLE", data['title'].to_list()[:10][j])
    print("\n")

SCORES [254.40851 253.18213 249.99191]
PREDICTION_by_title ['Спорт: Бейсбол' 'Игры: Киберспорт' 'Спорт: Дартс']
SAMPLE Шульский VS Дамил. Пояс. Мурад VS Черкасов. Гаджи VS Хадис. Бой? Т-34 и Джанго напали на Колтуна


SCORES [267.3344  261.73672 252.35751]
PREDICTION_by_title ['Спорт: Борьба' 'Спорт: Дартс' 'Спорт: Боевые искусства']
SAMPLE Тарасов VS Хейбати. Намитов VS Тарасов за пояс. Ершов VS Назир. Движ Борца VS Сушист. Савилов. Хизир


SCORES [278.13666 259.80603 259.1851 ]
PREDICTION_by_title ['Массовая культура: Смерти знаменитостей'
 'Еда и напитки: Пищевые аллергии'
 'Семья и отношения: Смерть родственников']
SAMPLE Мурад и Черкасов кровавое побоище. Мага Исма в шоке. КимЧи подрался с Фишером. Большой Папа Леденев.


SCORES [249.82545 246.67255 246.15733]
PREDICTION_by_title ['Спорт: Дартс' 'Спорт: Настольный теннис' 'Спорт: Крикет']
SAMPLE Шульский vs Вагабов. Тарасов vs Стоун.Сушист Фара Кузьминов Адамов Большой Папа vs Зулузиньо.Вадимыч


SCORES [251.88638 247.22667 243.46

#### Для формирования sample_submission будем брать только наилучшее предсказания для каждого видео
Сейчас у вас уже есть sample_submission с нужными для скоринга video_id, но пока нет информации о видео, она появится ближе к концу хакатона
Для примера прогоним через весь train dataset

In [8]:
topn=1
sample_submission = pd.DataFrame(data=data['video_id'].to_list(), columns=['video_id'])
sample_submission['predicted_tags']=np.nan
sample_submission['predicted_tags'] = sample_submission['predicted_tags'].astype('object')

for i, row in data.iterrows():
    scores, predictions = index.search(np.array([row['title_vector']]), topn)
    index_i = sample_submission[sample_submission.video_id == row.video_id].index
    sample_submission.at[index_i[0], 'predicted_tags'] = [tags_list[predictions[0][0]]] # вытаскиваем предсказание из 

In [9]:
print(sample_submission.head(5))


  video_id                                     predicted_tags
0  video_0                                   [Спорт: Бейсбол]
1  video_1                                    [Спорт: Борьба]
2  video_2          [Массовая культура: Смерти знаменитостей]
3  video_3                                     [Спорт: Дартс]
4  video_4  [Хобби и интересы: Игры и головоломки: Карточн...


#### В predicted_tags нужно записывать list тегов, например ['Карьера: Cтажировки', 'Карьера: Составление резюме'] или ['Массовая культура: Сериалы']

In [10]:
sample_submission.to_csv("sample_submission.csv", index_label=0)